In [1]:
import duckdb

In [2]:
with duckdb.connect("data/staging.db", read_only=True) as conn:

    conn.sql("SELECT * FROM shipments").show()

┌─────────────┬──────────┬────────────┬────────────┬─────────────────┬─────────────────┬──────────────┬────────────────────────┬──────────────────────┬───────────────┬─────────┬───────────────┬───────────────┬───────────────────┬────────────────┐
│ shipment_id │ order_id │ carrier_id │ service_id │ tracking_number │ shipping_status │ shipped_date │ expected_delivery_date │ actual_delivery_date │ shipping_cost │   op    │ emitted_ts_ms │     ts_ms     │ connector_version │ transaction_id │
│    int32    │  int32   │   int32    │   int32    │     varchar     │     varchar     │     date     │          date          │         date         │ decimal(10,2) │ varchar │     int64     │     int64     │      varchar      │    varchar     │
├─────────────┼──────────┼────────────┼────────────┼─────────────────┼─────────────────┼──────────────┼────────────────────────┼──────────────────────┼───────────────┼─────────┼───────────────┼───────────────┼───────────────────┼────────────────┤
│           

In [3]:
with duckdb.connect("data/staging.db", read_only=True) as conn:

    conn.sql("SELECT * FROM order_items").show()

┌───────────────┬──────────┬─────────┬──────────┬───────────────────┬──────────┬─────────┬───────────────┬───────────────┬───────────────────┬────────────────┬──────────┐
│ order_item_id │ order_id │ book_id │ quantity │ price_at_purchase │ discount │   op    │ emitted_ts_ms │     ts_ms     │ connector_version │ transaction_id │   lsn    │
│     int32     │  int32   │  int32  │  int32   │      double       │  double  │ varchar │     int64     │     int64     │      varchar      │    varchar     │  int64   │
├───────────────┼──────────┼─────────┼──────────┼───────────────────┼──────────┼─────────┼───────────────┼───────────────┼───────────────────┼────────────────┼──────────┤
│             1 │        1 │       1 │        1 │              8.99 │      0.0 │ c       │ 1746934319338 │ 1746934318997 │ 2.7.3.Final       │ 747            │ 24464600 │
│             2 │        1 │       2 │        2 │              9.99 │      1.0 │ c       │ 1746934319338 │ 1746934318997 │ 2.7.3.Final       │ 74

In [ ]:
with duckdb.connect("data/staging.db") as conn:

    conn.sql("DROP TABLE d_books")
    conn.sql("DROP TABLE d_carrier_services")
    conn.sql("DROP TABLE d_customers")
    conn.sql("DROP TABLE d_orders")
    conn.sql("DROP TABLE d_shipments")
    conn.sql("DROP TABLE f_book_inventory")
    conn.sql("DROP TABLE f_order_item_events")
    #conn.sql("DROP TABLE f_shipment_events")

In [45]:
conn.close()

In [51]:
import duckdb
import os

# === CONFIGURATION ===
SQL_FOLDER = "modeling/ctas"          # Folder containing .sql files
DUCKDB_FILE = "data/staging.db"       # DuckDB database file path

# === Connect to DuckDB ===
con = duckdb.connect(database=DUCKDB_FILE)

# === Get all .sql files ===
sql_files = sorted(
    [f for f in os.listdir(SQL_FOLDER) if f.endswith(".sql")]
)

# === Execute each SQL file ===
for sql_file in sql_files:
    path = os.path.join(SQL_FOLDER, sql_file)
    with open(path, "r") as file:
        sql = file.read()
        print(f"\n▶ Running: {sql_file}")
        try:
            result = con.execute(sql)
            # Optionally fetch results if the query returns rows
            if result.description:  # Check if there's a result set
                rows = result.fetchall()
                for row in rows:
                    print(row)
        except Exception as e:
            print(f"❌ Error in {sql_file}: {e}")

con.close()
print("\n✅ All queries executed.")


▶ Running: d_books.sql

▶ Running: d_carriers_services.sql

▶ Running: d_customers.sql

▶ Running: d_date.sql

▶ Running: d_orders.sql

▶ Running: d_shipments.sql

▶ Running: f_book_inventory.sql

▶ Running: f_order_item_events.sql
(4,)

▶ Running: f_shipment_events.sql

✅ All queries executed.


In [54]:
with duckdb.connect("data/staging.db", read_only=True) as conn:
    conn.sql("SELECT * FROM f_order_item_events").show()

┌──────────────────────────────────────┬──────────────────────────────────────┬─────────────┬──────────────────────────────────────┬─────────┬──────────────────────────────────┬──────────┬───────────────────┬──────────┐
│         order_item_event_sk          │               order_sk               │ customer_id │               book_sk                │   op    │ to_timestamp((src.ts_ms / 1000)) │ quantity │ price_at_purchase │ discount │
│                 uuid                 │                 uuid                 │    int32    │                 uuid                 │ varchar │     timestamp with time zone     │  int32   │      double       │  double  │
├──────────────────────────────────────┼──────────────────────────────────────┼─────────────┼──────────────────────────────────────┼─────────┼──────────────────────────────────┼──────────┼───────────────────┼──────────┤
│ dcd018bd-0d0a-476e-b237-0ca63e9858ae │ 8a59a1d7-e407-45b7-a328-4ad02e98255c │           3 │ 28e1cf45-bd91-41e2-917c-e2

In [79]:
with duckdb.connect("data/staging.db", read_only=True) as conn:
    conn.sql(
        """
        SELECT
            src.order_id,
            TO_TIMESTAMP (src.ts_ms / 1000) transaction_time,
            valid_to,
            valid_from,
            TO_TIMESTAMP (src.ts_ms / 1000) >= dor.valid_from valid_from_T,
            TO_TIMESTAMP (src.ts_ms / 1000) < dor.valid_to valid_to_T
        FROM
            order_items src
            INNER JOIN d_orders dor ON src.order_id = dor.order_id
        ORDER BY src.order_id, valid_from
            """
    ).show()

┌──────────┬────────────────────────────┬────────────────────────────┬────────────────────────────┬──────────────┬────────────┐
│ order_id │      transaction_time      │          valid_to          │         valid_from         │ valid_from_T │ valid_to_T │
│  int32   │  timestamp with time zone  │  timestamp with time zone  │  timestamp with time zone  │   boolean    │  boolean   │
├──────────┼────────────────────────────┼────────────────────────────┼────────────────────────────┼──────────────┼────────────┤
│        1 │ 2025-05-11 14:10:33.058+10 │ 2025-05-11 13:31:58.997+10 │ 2025-05-11 13:31:58.997+10 │ true         │ false      │
│        1 │ 2025-05-11 14:10:12.327+10 │ 2025-05-11 13:31:58.997+10 │ 2025-05-11 13:31:58.997+10 │ true         │ false      │
│        1 │ 2025-05-11 13:31:58.997+10 │ 2025-05-11 13:31:58.997+10 │ 2025-05-11 13:31:58.997+10 │ true         │ false      │
│        1 │ 2025-05-11 13:31:58.997+10 │ 2025-05-11 13:31:58.997+10 │ 2025-05-11 13:31:58.997+10 │ true

In [65]:
with duckdb.connect("data/staging.db", read_only=True) as conn:
    conn.sql(
        """
        SELECT
            to_timestamp(ts_ms / 1000)
        FROM
            order_items src
            """
    ).show()

┌──────────────────────────────┐
│ to_timestamp((ts_ms / 1000)) │
│   timestamp with time zone   │
├──────────────────────────────┤
│ 2025-05-11 13:31:58.997+10   │
│ 2025-05-11 13:31:58.997+10   │
│ 2025-05-11 13:31:58.997+10   │
│ 2025-05-11 13:31:59.028+10   │
│ 2025-05-11 13:31:59.028+10   │
│ 2025-05-11 13:31:59.028+10   │
│ 2025-05-11 13:31:59.052+10   │
│ 2025-05-11 13:31:59.052+10   │
│ 2025-05-11 13:31:59.064+10   │
│ 2025-05-11 14:10:03.288+10   │
│             ·                │
│             ·                │
│             ·                │
│ 2025-05-11 14:29:21.312+10   │
│ 2025-05-11 14:29:21.727+10   │
│ 2025-05-11 14:29:22.28+10    │
│ 2025-05-11 14:29:22.451+10   │
│ 2025-05-11 14:29:22.665+10   │
│ 2025-05-11 14:29:22.88+10    │
│ 2025-05-11 14:29:23.047+10   │
│ 2025-05-11 14:29:23.473+10   │
│ 2025-05-11 14:29:23.676+10   │
│ 2025-05-11 14:29:23.902+10   │
├──────────────────────────────┤
│     286 rows (20 shown)      │
└──────────────────────────────┘



In [64]:
with duckdb.connect("data/staging.db", read_only=True) as conn:
    conn.sql(
        """
        SELECT
            valid_to, valid_from
        FROM
            d_orders src
            """
    ).show()

┌────────────────────────────┬────────────────────────────┐
│          valid_to          │         valid_from         │
│  timestamp with time zone  │  timestamp with time zone  │
├────────────────────────────┼────────────────────────────┤
│ 2025-05-11 13:31:59.028+10 │ 2025-05-11 13:31:59.028+10 │
│ 2025-05-11 13:31:59.038+10 │ 2025-05-11 13:31:59.038+10 │
│ 2025-05-11 14:10:06.95+10  │ 2025-05-11 13:31:59.052+10 │
│ 2025-05-11 14:10:06.95+10  │ 2025-05-11 14:10:06.95+10  │
│ 2025-05-11 14:10:08.954+10 │ 2025-05-11 14:10:08.954+10 │
│ 2025-05-11 14:10:10.553+10 │ 2025-05-11 14:10:10.553+10 │
│ 2025-05-11 14:10:16.082+10 │ 2025-05-11 14:10:16.082+10 │
│ 2025-05-11 14:10:50.041+10 │ 2025-05-11 14:10:50.041+10 │
│ 2025-05-11 14:10:51.204+10 │ 2025-05-11 14:10:51.204+10 │
│ 2025-05-11 14:10:53.408+10 │ 2025-05-11 14:10:53.408+10 │
│             ·              │             ·              │
│             ·              │             ·              │
│             ·              │          